In [ ]:
##IMPORTANT NOTES ON HOW TO USE THIS

#You must create a file structor on the left as such

# Test_Dat
# |
# |------> Cube
# |
# L------> Cone

# You can place any number of sub-folders inside of the Test_Data folder, each folder inside will be set as a class, the class name being its folder name

# If you have more then 2 classes, you have to change the num_classes variable equal to the number of classes you are going to train the model on.

# These folders act as the label for whatever you place inside.

#Once done, run each code block below sequentially, and then save the model by right clicking the .h file on the left and click download.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from keras.preprocessing import image

#Note, for whatever reason I can't comment on the code block below or it error's out.
#The block below removes a hidden folder thats created by google colab inside the Test_Data causing the model to think there is +1 class.

In [ ]:
rm -rf `find -type d -name .ipynb_checkpoints`

In [ ]:
# This sets up the training data set. https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory
# The validation_split splits the data into a "training" and "validation" set that are randomly shuffled.
# The validation_split=0.2 sets the training set to 80% of the data within the class folders, and 20% for the validation set. The set
# validation_split data should bet set to the same number between the two sets.
# An important variable to tweak is batch_size. Search online what this variable does, and why you would want to tweak it.
# Note that you must have the batch size the same for the training and validation sets.
train_ds = tf.keras.utils.image_dataset_from_directory(
  "Test_Data",
  validation_split=0.2,
  shuffle=True,
  subset="training",
  seed=123433,
  image_size=(60, 60),
  batch_size=1)

Found 903 files belonging to 2 classes.
Using 723 files for training.


In [ ]:
# This code block sets up the validation set, the set that the model's accuracy is tested on.
val_ds = tf.keras.utils.image_dataset_from_directory(
  "Test_Data",
  validation_split=0.2,
  shuffle=True,
  subset="validation",
  seed=1212233,
  image_size=(60, 60),
  batch_size=1)
val_ds

Found 903 files belonging to 2 classes.
Using 180 files for validation.


<BatchDataset element_spec=(TensorSpec(shape=(None, 60, 60, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
#This just prints the class names tensorflow thinks you have. Its to double check everything is set up right, doesn't do anything useful.
class_names = train_ds.class_names
print(class_names)

['Cone', 'Cube']


In [ ]:
#The num_classes variable is for how many classes you are going to train. (If you change the number of classes you need to edit the python code as well. Look up the term logits to figure out how to use
# a trained model and get data out of it)
num_classes = 2

model = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=( 60, 60, 3)),
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(10, 2, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(15, 2, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(20, 2, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(40, 2, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [ ]:
#Compiles the model.
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
#Trains the model. Note: if you set the epochs too high you will overtrain the data, it will
# show a high accuracy, but when used to classify new data it will be far off.
# Right now (with around 250 images) the sweet spot seems to be between 3 and 5 epochs.
# The more data you have, you might be able to get away with larger epochs, especially if
# more layers are added to the model.
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=1,
  batch_size=1
)

723/723 [==============================] - 3s 5ms/step - loss: 0.1632 - accuracy: 0.9308 - val_loss: 0.1196 - val_accuracy: 0.9611


In [ ]:
#Saves the model. You still have to download the model to your computer using the file system on the left.
model.save('Cube_Cone_Model.h5') 4s 5ms/step - loss: 0.1865 - accuracy: 0.9308 - val_loss: 0.1526 - val_accuracy: 0.9500

In [ ]:
#Saves the model in a frozen state. It is much better for use with openVino's converter.
# You can download the model by mounting your google drive to the colab and dragging the whole save_model folder into your MyDrive folder.
# It will take forever to download from google colab.
tf.saved_model.save(model,"saved_model")


KeyboardInterrupt: ignored

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt


KeyboardInterrupt: ignored

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt


INFO:tensorflow:Assets written to: saved_model/assets


In [ ]:
#This mounts your google drive. It will prompt you to allow access to your google drive.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# This just prints the summary of the model below. Currently this model, once trained, is around 500mb.
model.summary()

Model: "sequential_51"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_47 (Rescaling)    (None, 60, 60, 3)         0         
                                                                 
 conv2d_185 (Conv2D)         (None, 59, 59, 10)        130       
                                                                 
 max_pooling2d_184 (MaxPooli  (None, 29, 29, 10)       0         
 ng2D)                                                           
                                                                 
 dropout_61 (Dropout)        (None, 29, 29, 10)        0         
                                                                 
 conv2d_186 (Conv2D)         (None, 28, 28, 15)        615       
                                                                 
 max_pooling2d_185 (MaxPooli  (None, 14, 14, 15)       0         
 ng2D)                                               

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
rm -rf saved_modell

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 60, 60, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 59, 59, 64)        832       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 29, 29, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        8224      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 13, 256)       3